1. Выбираем часть датасета где acnum = VQ-BDQ, pos = 1, phase = TAKEOFF

In [46]:
import pandas as pd

egtm_raw = pd.read_csv('./EGTM.csv', parse_dates=['RECORDED_DT'])

In [47]:
n1_raw = pd.read_csv('./Data N1.csv', parse_dates=['RECORDED_DT'])

n1_cols = ['acnum', 'pos', 'recordedts', 'n1_delt']

n1_clean = n1_raw.rename(columns={
  'AIRCRAFT': 'acnum',
  'POSITION': 'pos',
  'RECORDED_DT': 'recordedts',
  'DELTA N1 (Actual vs Cmd)': 'n1_delt'
})[n1_cols]

n1_clean['recordedts'] = pd.to_datetime(n1_clean['recordedts'], format='mixed')


n1 = n1_clean[(n1_clean['acnum'] == 'VQ-BDQ') & (n1_clean['pos'] == 1)].copy()

n1.notna().sum()

acnum         1352
pos           1352
recordedts    1352
n1_delt       1247
dtype: int64

In [48]:
n2_raw = pd.read_csv('./Data N2.csv', parse_dates=['RECORDED_DT'])

n2_cols = ['acnum', 'pos', 'recordedts', 'n2_delt']

n2_clean = n2_raw.rename(columns={
  'AIRCRAFT': 'acnum',
  'POSITION': 'pos',
  'RECORDED_DT': 'recordedts',
  'N2 Delt': 'n2_delt'
})[n2_cols]

n2_clean['recordedts'] = pd.to_datetime(n2_clean['recordedts'], format='mixed')


n2 = n2_clean[(n2_clean['acnum'] == 'VQ-BDQ') & (n2_clean['pos'] == 1)].copy()

n2.notna().sum()

acnum         1294
pos           1294
recordedts    1294
n2_delt       1247
dtype: int64

In [49]:
egtm_cols = ['acnum', 'pos', 'recordedts', 'egtm']

egtm_clean = egtm_raw.rename(columns={
  'AIRCRAFT': 'acnum',
  'POSITION': 'pos',
  'RECORDED_DT': 'recordedts',
  'RWCTOMAR': 'egtm'
})[egtm_cols]

egtm_clean['recordedts'] = pd.to_datetime(egtm_clean['recordedts'], format='mixed')
egtm_clean['reportts'] = egtm_clean['recordedts'] 

egtm = egtm_clean[(egtm_clean['acnum'] == 'VQ-BDQ') & (egtm_clean['pos'] == 1)].copy()

egtm.tail(3)

,acnum,pos,recordedts,egtm,reportts
1281,VQ-BDQ,1,2020-02-02 04:14:00,106.549,2020-02-02 04:14:00
1282,VQ-BDQ,1,2020-02-02 16:24:00,106.186,2020-02-02 16:24:00
1283,VQ-BDQ,1,2020-02-02 22:24:00,106.221,2020-02-02 22:24:00


In [50]:
egtm = pd.merge(egtm, n1, on=['acnum', 'pos', 'recordedts'], how='left')
egtm = pd.merge(egtm, n2, on=['acnum', 'pos', 'recordedts'], how='left')

In [51]:
egtm

,acnum,pos,recordedts,egtm,reportts,n1_delt,n2_delt
0,VQ-BDQ,1,2018-05-20 19:36:47,132.144,2018-05-20 19:36:47,NaN,NaN
1,VQ-BDQ,1,2018-05-22 01:48:50,127.592,2018-05-22 01:48:50,NaN,NaN
2,VQ-BDQ,1,2018-05-23 19:51:23,128.712,2018-05-23 19:51:23,NaN,NaN
3,VQ-BDQ,1,2018-05-25 01:07:05,129.118,2018-05-25 01:07:05,NaN,NaN
4,VQ-BDQ,1,2018-05-26 10:41:08,129.008,2018-05-26 10:41:08,NaN,NaN
...,...,...,...,...,...,...,...
1279,VQ-BDQ,1,2020-01-31 14:36:22,106.804,2020-01-31 14:36:22,-0.090004,-0.290552
1280,VQ-BDQ,1,2020-01-02 14:42:00,106.877,2020-01-02 14:42:00,-0.059998,-0.264996
1281,VQ-BDQ,1,2020-02-02 04:14:00,106.549,2020-02-02 04:14:00,-0.090004,-0.220833
1282,VQ-BDQ,1,2020-02-02 16:24:00,106.186,2020-02-02 16:24:00,0.160004,-0.232732


In [52]:
neo_raw = pd.read_csv(
  './a320_a321_neo_full_acms_parameters.csv', 
  parse_dates=['reportts', 'processedts'], 
  low_memory=False
)
neo = neo_raw[
  (neo_raw['acnum'] == 'VQ-BDQ') & 
  (neo_raw['pos'] == 1) & 
  (neo_raw['reportname'] == 'A320 ENGINE TAKE OFF REPORT')
].copy()

neo.tail(3)

,acnum,reportname,reportts,file_path,pos,fltdes,dep,arr,pf,wai,...,tr,ttp,vb1,vb2,vorrc,vorv,votm,vsva,w14,processedts
41207,VQ-BDQ,A320 ENGINE TAKE OFF REPORT,2025-04-20 12:55:21,adw-data/aircraft/pdm/data/qar/binary/A320NEO/...,1,3212,LUKK,UUDD,0.45,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-09-24 02:44:46.730
41225,VQ-BDQ,A320 ENGINE TAKE OFF REPORT,2025-04-20 19:00:31,adw-data/aircraft/pdm/data/qar/binary/A320NEO/...,1,3033,UUDD,UIUU,0.43,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-09-24 02:44:46.917
41237,VQ-BDQ,A320 ENGINE TAKE OFF REPORT,2025-04-21 01:43:58,adw-data/aircraft/pdm/data/qar/binary/A320NEO/...,1,3034,UIUU,UUDD,0.42,0.0,...,NaN,46.0,0.2,0.2,52.9,NaN,NaN,0.77,1214.0,2023-09-24 02:44:47.137


In [53]:
def merge_by_date(left: pd.DataFrame, right: pd.DataFrame):
  left.sort_values('reportts', inplace=True)
  right.sort_values('reportts', inplace=True)

  left = neo.set_index('reportts')
  right = egtm.set_index('reportts')

  right_indexed = \
      right.reindex(left.index, method='nearest').reset_index()
  return pd.merge(left, right_indexed, on='reportts', suffixes=(None, '_'))

In [54]:

merged = merge_by_date(neo, egtm)

merged[['reportts','acnum','pos','acnum_','pos_','recordedts','egtm', 'n1_delt', 'n2_delt']]


,reportts,acnum,pos,acnum_,pos_,recordedts,egtm,n1_delt,n2_delt
0,2018-05-20 02:58:38,VQ-BDQ,1,VQ-BDQ,1,2018-05-20 19:36:47,132.144,NaN,NaN
1,2018-05-20 19:36:46,VQ-BDQ,1,VQ-BDQ,1,2018-05-20 19:36:47,132.144,NaN,NaN
2,2018-05-22 01:48:49,VQ-BDQ,1,VQ-BDQ,1,2018-05-22 01:48:50,127.592,NaN,NaN
3,2018-05-23 19:51:22,VQ-BDQ,1,VQ-BDQ,1,2018-05-23 19:51:23,128.712,NaN,NaN
4,2018-05-25 01:07:04,VQ-BDQ,1,VQ-BDQ,1,2018-05-25 01:07:05,129.118,NaN,NaN
...,...,...,...,...,...,...,...,...,...
2249,2025-04-20 00:51:22,VQ-BDQ,1,VQ-BDQ,1,2020-12-01 13:11:00,106.968,-0.079994,-0.267799
2250,2025-04-20 08:53:20,VQ-BDQ,1,VQ-BDQ,1,2020-12-01 13:11:00,106.968,-0.079994,-0.267799
2251,2025-04-20 12:55:21,VQ-BDQ,1,VQ-BDQ,1,2020-12-01 13:11:00,106.968,-0.079994,-0.267799
2252,2025-04-20 19:00:31,VQ-BDQ,1,VQ-BDQ,1,2020-12-01 13:11:00,106.968,-0.079994,-0.267799


In [55]:
diff = (merged['reportts'] - merged['recordedts']) \
  .astype('timedelta64[s]').astype('int')

In [56]:
merged_closest = \
  merged[(diff >= (-1000 * 60)) & (diff <= (1000 * 60 * 60))] \
    .drop(columns=['file_path', 'reportname', 'processedts', 'acnum_', 'pos_'])


In [57]:
merged_closest.to_csv('./sample-VQ-BDQ-1-egtm.csv', index=False)